In [4]:
#Importing Libraries
import numpy as np
import random
from numpy import inf

In [11]:
def cost(m, n, f, D, route, dist_cost):
    # Repeat the amount of times of ant paths
    for i in range(m):
            
            cost = 0 # Cost start for each ant
            current = np.random.permutation(np.arange(n)) # This is permutation for flow matrix (p vector)
            
            for j in range(n-1):
                # Working out the cost of the entire route
                cost += D[int(route[i,j])-1,int(route[i,j+1])-1] * f[current[int(route[i,j])-1]][current[int(route[i,j+1])-1]]
            
            # Setting the dist cost for each ant here then returning it
            dist_cost[i]=cost
    return dist_cost

In [6]:
def init_phero_matrix(n):
    # Matrix of random number between 0 to 1 in a n x n matrix
    p = np.random.uniform(0, 1, size=n*n).reshape(n,n)
    return p

In [16]:
def prob(d, current, p):
    combine = np.zeros(n) # Array of zeros 
    cummulative = np.zeros(n) # Array of zeros
    
    d[:,current] = 0     # Making Visiblity of the current city as 0
                
    df = np.power(d[current,:],alpha)  # Calculate Visibility 
    pf = np.power(p[current,:],beta)         # Calculate Pheromones
            
    pf = pf[:,np.newaxis] # Converts a matrix from a 1D array to a 2D array     
    df = df[:,np.newaxis] # Converts a matrix from a 1D array to a 2D array 
    combine = np.multiply(pf,df) # Multiplies the features 
    total = np.sum(combine) # Gets sum total of combine
                
    probs = combine/total # Finding probability of element probs(i) = combine(i)/total

    cummulative = np.cumsum(probs) # Calculating cummulative sum
    
    return cummulative

In [8]:
def evapourate(p, n, e, m, route, dist):
    p = (1-e)*p # Pheromone evapouration
    for i in range(m):
            for j in range(n-1):
                fitness = 1/dist[i] # Fitness Function
                # Depositing using the fitness function
                p[int(route[i,j])-1,int(route[i,j+1])-1] = p[int(route[i,j])-1,int(route[i,j+1])-1] + fitness
    return p

In [29]:
def search(m, e, term, n, D, f, beta, alpha):
    np.seterr(divide='ignore', invalid='ignore') # Gets rid of error which doesn't affect code
    p = init_phero_matrix(n) # Initialise pheromones
    route = np.ones((m,n+1)) # Makes a m x n+1 matrix the +1 is for starting position (filled with 1s)
    
    # Starts the loop
    for ite in range(term):
        route[:,0] = 1 # To make sure the start and the end is 1
        for i in range(m):
            dis = np.array(D) # Copy of distance matrix
            for j in range(n-1):
                
                current = int(route[i,j]-1) # Current City of the Ant
                cummulative = prob(dis, current, p) # Probabilities
                ran = np.random.random_sample() # Randon number between 0 and 1
        
                # This is to stop it from dividing arrays with 0 items 
                try:
                    # Finding the next city having probability higher than r
                    city = np.nonzero(cummulative>ran)[0][0]+1 
                except IndexError:
                    pass
                
                route[i,j+1] = city # Adding city to route 
                
            left = list(set([i for i in range(1,n+1)])-set(route[i,:-2]))[0] # Finding what cities are left
            route[i,-2] = left # Adding untraversed city to route
        
        optimum = np.array(route) # Intializing optimal route
        dist_cost = np.zeros((m,1)) # Intializing total distance with zero 
        
        # Cost
        c = cost(m, n, f, D, optimum, dist_cost)
        
        min_loc = np.argmin(c) # Finding location of minimum of distance cost
        min_cost = c[min_loc] # Finding minimum of distance cost
        best_route = route[min_loc,:] # Intializing current traversed as best route
        
        # Evapouration and Pheromone deposit
        p = evapourate(p, n, e, m, optimum, c)
    
    # Best path cost calculation
    min_cost = int(min_cost[0]) + D[int(best_route[-2])-1,0]
    
    return best_route, min_cost, optimum

In [32]:
# Initial Variables and Start
ants = 100 # m
evaporation = 0.9 # e
iterations = 100 # For termination
alpha = 1 # Pheromone Factor
beta = 2 # Visiblity Factor

fitness_eva = ants * iterations # Work out fitness eva before starting to confirm that numbers are right for trials

# Loading the file
filename = 'Uni50a.dat'

# Gets the whole two matrices together skips first number in the file
data = np.genfromtxt(filename, dtype=int, skip_header=1)

# This gets the first number which is for the n
with open(filename, "r") as fp:
    lines = fp.readlines()
    first = lines[0].split(',')[0]

n = int(first) # 50
matrix = np.split(data, 2) # Splits matrix in two

# Distance and Flow matrix
D = matrix[0]
f = matrix[1]

# Double check if it is 10000 fitness evaulations
if(fitness_eva == 10000):
    # Searches with ACO
    best, minc, opt = search(ants, evaporation, iterations, n, D, f, beta, alpha)
    # Prints all the answers
    print('Route of all the ants at the end :\n')
    print(opt)
    print('\nBest path :', best)
    print('\nCost of the best path', minc)
else:
    print("Choose Values for fitness evaluations to equal 10000")

Route of all the ants at the end :

[[ 1.  4. 18. ... 26. 14.  1.]
 [ 1. 30. 35. ... 23.  6.  1.]
 [ 1. 34. 33. ...  3. 45.  1.]
 ...
 [ 1. 34. 10. ... 45. 36.  1.]
 [ 1. 22. 43. ... 45.  3.  1.]
 [ 1. 34. 46. ... 40.  8.  1.]]

Best path : [ 1. 32. 46. 22. 24. 10. 23. 44.  6. 16. 41. 11. 17. 36. 40. 21.  7.  2.
 25. 37. 12. 20. 33. 47. 19. 15. 42. 30. 35. 49. 38. 29.  4. 26. 50. 34.
  8. 45. 28. 39. 27. 18.  3. 13. 31. 43. 48.  5. 14.  9.  1.]

Cost of the best path 95900
